# Semantic Similarity - Experiment 01
The objective of this trial is to expand the SCA_index (i.e., Semantic Content Analysis Index) to a full word embedding, setting a subjective or objective load for each word.

## Introduction

### Libraries

In [1]:
## Data analysis packages:
import pandas as pd
import numpy as np
from math import isnan  #Verifies if a given value is numerical.

In [2]:
## Visualization packages:
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Definitions

In [3]:
## Forcing Pandas to display any number of elements
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', None)

In [4]:
# ## Based on: https://stackoverflow.com/questions/25351968/how-can-i-display-full-non-truncated-dataframe-information-in-html-when-conver
# def print_full(x):
#     pd.set_option('display.max_rows', None)
#     pd.set_option('display.max_columns', None)
#     pd.set_option('display.width', 2000)
#     pd.set_option('display.float_format', '{:20,.2f}'.format)
#     pd.set_option('display.max_colwidth', None)
#     print(x)
#     pd.reset_option('display.max_rows')
#     pd.reset_option('display.max_columns')
#     pd.reset_option('display.width')
#     pd.reset_option('display.float_format')
#     pd.reset_option('display.max_colwidth')

## Exploring the SpaCy Word Embeddings: 
Also using Spacy library: https://spacy.io/
> !pip install -U spacy  
> !python -m spacy download en_core_web_sm  
> !python -m spacy download en_core_web_lg

Some instructions on how to use it:  
https://spacy.io/usage/spacy-101

In [5]:
## Importing SpaCy library:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_lg")

In [6]:
## We check below that this model has 514.157 keys and vectors, respectively.
nlp.meta['vectors']

{'width': 300,
 'vectors': 514157,
 'keys': 514157,
 'name': 'en_vectors',
 'mode': 'default'}

In [7]:
## Again, checking the number of keys.
nlp.vocab.vectors.n_keys

514157

In [8]:
## Getting the word embedding: data (i.e., the matrix containing the vector values for each word)
word_embedding = nlp.vocab.vectors.data

## Verifying the shape of the word embedding matrix:
word_embedding.shape

(514157, 300)

--- 
### Finding the words associated with the embedding:

In [39]:
## Extracting the words associated with each index:
index = nlp.vocab.vectors.keys()
words_associated = [nlp.vocab[i].text for i in index]

In [10]:
## Checking the word in position 514156, wich is "Lahouaiej":
words_associated[514156]

'Lahouaiej'

In [11]:
## Finding the respective row (index) for a given word:
rows = nlp.vocab.vectors.find(keys=["cat", "dog", "Lahouaiej"])
rows

array([  3201,   1147, 514156], dtype=int32)

---
## SCA - Glasgow Norms
* Read the SCA from Glasgow Norms;  
* Import F_s and F_o from the previous study;  
* Train the MLP classifier.

In [25]:
df_factors = pd.read_csv('../data/df_factors.csv', sep=';')
df_factors.head()

,words,F_Objectivity,F_Subjectivity,F_Context
0,abattoir,0.512527,0.380603,0.960466
1,abbey,0.714765,0.240456,0.696198
2,abbreviate,0.286952,0.171052,0.767043
3,abdicate,0.144736,0.384300,0.863127
4,abdication,0.167654,0.334086,0.896733


In [26]:
# ### Selecionando apenas as palavras no df_factors que atendam aos critérios:
# df_selected = df_factors.loc[((df_factors['F_Subjectivity'] > 0.75) | (df_factors['F_Subjectivity'] < 0.3)) & ((df_factors['F_Objectivity'] > 0.75) | (df_factors['F_Objectivity'] < 0.3))]


In [27]:
# df_factors = df_selected.copy()

In [28]:
SCA_words = [word for word in df_factors.words]

In [29]:
SCA_embedding_rows =  nlp.vocab.vectors.find(keys=SCA_words)

In [30]:
len(SCA_embedding_rows)

5553

> Separating the SCA-GlasgowNorms data into train and test:

In [31]:
from sklearn.model_selection import train_test_split

# Separar os dados em conjuntos de treino (70%) e teste (30%)
train_df, test_df = train_test_split(df_factors, test_size=0.2, random_state=42)

In [19]:
# Função para criar os conjuntos de treino e resposta
def create_data(dataframe):
    X = {}
    Y = {}
    
    for index, row in dataframe.iterrows():
        word = row['words']
        f_objectivity = row['F_Objectivity']
        f_subjectivity = row['F_Subjectivity']

        if word in nlp.vocab:
            indice = nlp.vocab.strings[word]
            vetor_embedding = word_embedding[index]
            X[word] = vetor_embedding
            Y[word] = {'F_Objectivity': f_objectivity, 'F_Subjectivity': f_subjectivity}

    return pd.DataFrame.from_dict(X, orient='index'), pd.DataFrame.from_dict(Y, orient='index')

In [45]:
# Função para criar os conjuntos de treino e resposta
def create_data_debug(dataframe):
    X = {}
    Y = {}
    
    for index, row in dataframe.iterrows():
        word = row['words']
        f_objectivity = row['F_Objectivity']
        f_subjectivity = row['F_Subjectivity']

        if word in nlp.vocab:
            idx = nlp.vocab.strings[word]
            vetor_embedding = nlp.vocab[idx].vector
            X[word] = vetor_embedding
            Y[word] = {'F_Objectivity': f_objectivity, 'F_Subjectivity': f_subjectivity}
            print(f'-- DEBUG:\nIndex: {index}\nIndice: {idx}\nWord: {word}\n\n')

    return pd.DataFrame.from_dict(X, orient='index'), pd.DataFrame.from_dict(Y, orient='index')

In [44]:
nlp.vocab.strings['pudding']

3403131918490088723

In [ ]:
nlp.vocab[3403131918490088723].vector

In [46]:
## Debugging:
Xbug_test, Ybug_test = create_data_debug(test_df)

-- DEBUG:
Index: 3865
Indice: 3403131918490088723
Word: pudding


-- DEBUG:
Index: 2801
Indice: 720313458719916916
Word: letter


-- DEBUG:
Index: 230
Indice: 13605738838253397229
Word: aroused


-- DEBUG:
Index: 3361
Indice: 13398675276606405380
Word: only


-- DEBUG:
Index: 2924
Indice: 4855781600843238028
Word: lunching


-- DEBUG:
Index: 1964
Indice: 527094160297002804
Word: folk


-- DEBUG:
Index: 3575
Indice: 10686358792985228098
Word: persist


-- DEBUG:
Index: 3221
Indice: 14565421918427191397
Word: national


-- DEBUG:
Index: 2022
Indice: 11216980060904606566
Word: freeze


-- DEBUG:
Index: 5146
Indice: 5714931530947501195
Word: trim


-- DEBUG:
Index: 439
Indice: 922631143178789688
Word: betray


-- DEBUG:
Index: 2759
Indice: 12601672956933186127
Word: lawyer


-- DEBUG:
Index: 373
Indice: 3119909018940161727
Word: bass


-- DEBUG:
Index: 4882
Indice: 16190201684402242524
Word: symptom


-- DEBUG:
Index: 4035
Indice: 6003327276716044531
Word: relate


-- DEBUG:
Index: 576
Ind

In [48]:
# Creating train and test datasets:
X_train, Y_train = create_data_debug(train_df)
X_test, Y_test = create_data_debug(test_df)

# Exibir as dimensões dos conjuntos de treino e teste
print("Train data dimension:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

print("\nTest data dimension:")
print("X_test:", X_test.shape)
print("Y_test:", Y_test.shape)

-- DEBUG:
Index: 3565
Indice: 16752602526984977479
Word: periodical


-- DEBUG:
Index: 2572
Indice: 3166149224535108880
Word: intellect


-- DEBUG:
Index: 1253
Indice: 6807689115566132187
Word: culture


-- DEBUG:
Index: 1657
Indice: 4467559365180225526
Word: emancipation


-- DEBUG:
Index: 831
Indice: 17889400642917665921
Word: chasm


-- DEBUG:
Index: 2784
Indice: 17067105940613322182
Word: legend


-- DEBUG:
Index: 1335
Indice: 5441004699069252923
Word: decree


-- DEBUG:
Index: 2846
Indice: 2025327972831713556
Word: linear


-- DEBUG:
Index: 238
Indice: 14002191346934931615
Word: arse


-- DEBUG:
Index: 2617
Indice: 5481122766517169115
Word: jaguar


-- DEBUG:
Index: 3768
Indice: 18237595148359232214
Word: practice


-- DEBUG:
Index: 2505
Indice: 7936449425443571492
Word: impulse


-- DEBUG:
Index: 3240
Indice: 14014441641201808195
Word: neglect


-- DEBUG:
Index: 2974
Indice: 15691136892911553773
Word: mansion


-- DEBUG:
Index: 1001
Indice: 4303796670756400024
Word: combat


-- D

In [49]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
periodical,-1.627200,-0.782380,1.666100,0.64415,5.802500,-3.86060,2.11450,2.246400,0.11389,-0.57345,5.73660,2.68970,-2.9695,2.05920,1.17530,2.126600,4.70350,2.73360,-3.58720,1.118700,-0.33484,-1.407100,-4.75610,2.23020,-2.05900,-0.144110,-1.90420,-3.21300,-0.34878,1.15970,-1.340600,0.47455,0.76463,0.62292,-2.395900,-0.086495,1.158500,-0.680910,1.14940,1.32050,1.89910,-0.035937,-0.36217,2.45830,1.10640,2.30780,-0.640620,0.28873,-1.60050,-2.39470,-2.21640,3.73500,-0.78451,-0.50572,-2.403900,1.987700,-2.20640,2.12540,2.011200,-3.42740,0.386360,-0.990080,-2.24970,1.62110,3.09440,0.64471,1.432100,-0.62388,0.867700,1.11960,-1.925100,-2.90010,-2.830300,-2.88330,-0.33352,3.17870,-1.47500,1.181100,-1.86000,0.123780,-1.649600,-0.14419,-0.56384,1.13990,2.22990,3.66950,-1.08090,-2.70110,2.361700,-2.088400,-0.71105,1.610800,3.72370,-1.2607,2.37400,-1.998000,1.720000,1.45120,-0.399680,4.17960,2.69910,2.52750,2.08680,1.219600,-1.02390,3.43800,3.123000,-3.86070,1.48550,-4.18480,3.79030,0.88335,-2.85300,-0.98828,1.94630,1.176900,-1.83060,2.754000,-1.781200,-1.47680,0.84724,-4.41080,1.74440,1.40660,1.85590,-4.2074,-0.42883,-2.28530,2.61080,-2.44520,-1.565800,0.60370,2.07170,2.74070,0.50984,0.82838,-1.914900,-1.22880,0.70129,-3.16500,-0.403730,-1.53140,-0.564260,1.63540,3.8474,-0.94437,-0.99321,-0.052637,0.75979,3.275000,2.39540,0.05155,0.777010,1.33140,-1.79120,0.54084,-0.39345,-0.945120,-1.19330,-1.52460,-1.97940,1.4443,1.13990,3.445800,-2.80270,-1.21350,-6.362900,1.643800,-0.24045,0.16754,1.418400,0.92152,2.77860,0.41264,2.33820,3.16470,1.26470,1.10720,-3.23810,-1.25910,-0.83185,-1.605900,0.57993,-2.11800,-0.54878,0.76078,-0.074174,0.30904,-0.28318,1.759700,-0.83301,-2.57780,-3.72420,0.58209,3.82490,1.47940,-2.24200,-2.071500,0.82211,3.48490,-1.61720,-4.01410,-0.16727,-2.02820,7.086500,0.84765,-3.49450,1.27650,2.29560,0.475230,2.31430,-0.93912,0.030626,-1.464700,-0.42383,1.79430,-0.78176,-3.80790,0.45426,0.584070,-0.151750,2.42480,-3.875900,-0.19335,-3.51550,-1.390700,-1.231900,0.48635,1.660200,0.84975,4.61750,-0.81367,0.47439,1.677400,1.73930,4.31650,0.14046,1.41820,-1.87130,-1.494800,-1.02730,-1.264000,2.511700,2.146300,-1.05200,2.575400,-2.890900,2.070600,-0.56698,2.65870,2.59990,-4.95480,-2.07780,-0.67968,-0.62967,-2.59250,-1.57080,-2.25200,-0.10327,1.25720,-2.79040,2.89630,3.159800,2.28880,3.2515,-0.41810,2.101200,2.73860,-0.14320,-0.355010,-1.738700,1.54270,1.733800,-2.47970,1.041200,1.845200,-0.012340,-1.63990,-1.290000,-0.055819,3.78240,0.870420,3.85640,0.43643,0.13927,-2.903700,0.46992,0.76273,0.524870,-2.51860,2.55550,-1.15170,-1.55030,-2.03400,-2.33070,-0.35606,0.241200,-5.179000,-3.23970,-1.40730
intellect,-0.154690,1.676800,-0.351630,-0.57609,2.088700,2.61040,0.66495,2.229900,-4.99240,-1.65200,4.49340,3.32340,-1.5071,2.74590,1.54030,1.224500,1.71150,0.29566,-1.08720,-0.314340,1.09350,0.442640,-4.21780,-2.21770,-0.33491,-2.498900,-2.99290,-0.89613,-1.53340,2.52

In [50]:
Y_train

,F_Objectivity,F_Subjectivity
periodical,0.316950,0.157740
intellect,0.166311,0.756693
culture,0.368721,0.672379
emancipation,0.126689,0.368884
chasm,0.450759,0.459234
...,...,...
prawn,0.952891,0.110865
tweezers,0.953419,0.193659
university,0.835052,0.695757
wasteful,0.200742,0.492842


> Comparing X_test with Xbug_test

In [51]:
X_test == Xbug_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
pudding,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
letter,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True

#### Binarizing Y_train and Y_test
Once we run the first MLP model, the performance wasn't over 54%. 
In this Section, we will binarize the semantic factor values following the median values.

In [23]:
# Binarizing through list comprehension
Y_train['F_Objectivity'] = ['high' if f_objectivity >= 0.565 else 'low' for f_objectivity in Y_train['F_Objectivity']]
Y_train['F_Subjectivity'] = ['high' if f_subjectivity >= 0.392 else 'low' for f_subjectivity in Y_train['F_Subjectivity']]

Y_test['F_Objectivity'] = ['high' if f_objectivity >= 0.565 else 'low' for f_objectivity in Y_test['F_Objectivity']]
Y_test['F_Subjectivity'] = ['high' if f_subjectivity >= 0.392 else 'low' for f_subjectivity in Y_test['F_Subjectivity']]

In [52]:
# Binarizing through list comprehension
Y_train['F_Objectivity'] = [1 if f_objectivity >= 0.565 else 0 for f_objectivity in Y_train['F_Objectivity']]
Y_train['F_Subjectivity'] = [1 if f_subjectivity >= 0.392 else 0 for f_subjectivity in Y_train['F_Subjectivity']]

Y_test['F_Objectivity'] = [1 if f_objectivity >= 0.565 else 0 for f_objectivity in Y_test['F_Objectivity']]
Y_test['F_Subjectivity'] = [1 if f_subjectivity >= 0.392 else 0 for f_subjectivity in Y_test['F_Subjectivity']]

In [53]:
Y_train

,F_Objectivity,F_Subjectivity
periodical,0,0
intellect,0,1
culture,0,1
emancipation,0,0
chasm,0,1
...,...,...
prawn,1,0
tweezers,1,0
university,1,1
wasteful,0,1


---
### Training a MLP Classifier for word semantic content

In [54]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [55]:
# Define a new MLP architecture
model = Sequential([
    Dense(256, activation='relu', input_shape=(300,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # 2 neurons for binary classification with softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use categorical_crossentropy for categorical data
              metrics=['accuracy'])

In [56]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               77056     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 2)                 66        
                                                                 
Total params: 120354 (470.13 KB)
Trainable params: 120354 (470.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


> Como converti a saída em dados categóricos, é preciso antes utilizar OneHotEncoder:

In [57]:
## Como já foram transformados anteriormente em [0,1], preciso apenas torná-los como lista:
Y_train_array = Y_train.to_numpy()
Y_test_array = Y_test.to_numpy()

X_train_array = X_train.to_numpy()
X_test_array = X_test.to_numpy()

# Print the first few elements to verify
print(Y_train_array[:5])  # Print the first 5 elements


[[0 0]
 [0 1]
 [0 1]
 [0 0]
 [0 1]]


In [58]:
# Convert multilabel categorical labels to binary vectors
multi_label_binarizer = MultiLabelBinarizer()
Y_train_encoded = multi_label_binarizer.fit_transform(Y_train)
Y_test_encoded = multi_label_binarizer.transform(Y_test)

In [59]:
len(Y_train_encoded)

2

In [60]:
Y_train_encoded

array([[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [61]:
Y_train

,F_Objectivity,F_Subjectivity
periodical,0,0
intellect,0,1
culture,0,1
emancipation,0,0
chasm,0,1
...,...,...
prawn,1,0
tweezers,1,0
university,1,1
wasteful,0,1


In [62]:
X_train_array

array([[-1.6272  , -0.78238 ,  1.6661  , ..., -5.179   , -3.2397  ,
        -1.4073  ],
       [-0.15469 ,  1.6768  , -0.35163 , ..., -0.12766 , -5.476   ,
         5.2349  ],
       [-0.025891, -2.0093  ,  0.49408 , ..., -1.2505  , -7.2527  ,
         5.329   ],
       ...,
       [-1.409   ,  0.093909,  0.17297 , ..., -0.54633 ,  0.59882 ,
         0.67005 ],
       [-1.105   , -0.3351  , -1.8275  , ...,  0.035378, -3.728   ,
         1.1732  ],
       [-0.92249 ,  2.2266  , -7.5058  , ...,  2.9505  , -5.3231  ,
         2.5729  ]], dtype=float32)

In [63]:
Y_train

,F_Objectivity,F_Subjectivity
periodical,0,0
intellect,0,1
culture,0,1
emancipation,0,0
chasm,0,1
...,...,...
prawn,1,0
tweezers,1,0
university,1,1
wasteful,0,1


In [64]:
# Train the model
history = model.fit(X_train_array, Y_train_array, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50


188/188 [==============================] - 1s 3ms/step - loss: 0.4673 - accuracy: 0.7414 - val_loss: 0.4055 - val_accuracy: 0.7939
Epoch 2/50
188/188 [==============================] - 0s 2ms/step - loss: 0.3130 - accuracy: 0.7837 - val_loss: 0.4125 - val_accuracy: 0.7606
Epoch 3/50
188/188 [==============================] - 0s 2ms/step - loss: 0.2317 - accuracy: 0.7854 - val_loss: 0.4517 - val_accuracy: 0.8112
Epoch 4/50
188/188 [==============================] - 0s 2ms/step - loss: 0.1732 - accuracy: 0.7970 - val_loss: 0.4799 - val_accuracy: 0.7753
Epoch 5/50
188/188 [==============================] - 0s 2ms/step - loss: 0.1286 - accuracy: 0.7960 - val_loss: 0.5900 - val_accuracy: 0.7673
Epoch 6/50
188/188 [==============================] - 0s 2ms/step - loss: 0.1041 - accuracy: 0.8020 - val_loss: 0.6119 - val_accuracy: 0.7553
Epoch 7/50
188/188 [==============================] - 0s 2ms/step - loss: 0.0871 - accuracy: 0.7950 - val_loss: 0.6361 - val_accuracy: 0.7846
Epoc

In [65]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_array, Y_test_array)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

29/29 [==============================] - 0s 1ms/step - loss: 1.9241 - accuracy: 0.7736
Test Accuracy: 77.36%


In [66]:
# Treina o modelo
history = model.fit(X_train, Y_train,
                    epochs=50,
                    batch_size=32,
                    validation_split=0.1)  # Usamos parte dos dados de treino como validação

# Avalia o modelo com os dados de teste
loss, accuracy = model.evaluate(X_test.values, Y_test.values)
print(f"Acurácia do modelo nos dados de teste: {accuracy * 100:.2f}%")

Epoch 1/50
106/106 [==============================] - 0s 3ms/step - loss: 0.2633 - accuracy: 0.7986 - val_loss: 0.4547 - val_accuracy: 0.7500
Epoch 2/50
106/106 [==============================] - 0s 2ms/step - loss: 0.0974 - accuracy: 0.8146 - val_loss: 0.5949 - val_accuracy: 0.8059
Epoch 3/50
106/106 [==============================] - 0s 3ms/step - loss: 0.0341 - accuracy: 0.8033 - val_loss: 0.7319 - val_accuracy: 0.7606
Epoch 4/50
106/106 [==============================] - 0s 2ms/step - loss: 0.0283 - accuracy: 0.7971 - val_loss: 0.8396 - val_accuracy: 0.7553
Epoch 5/50
106/106 [==============================] - 0s 2ms/step - loss: 0.0268 - accuracy: 0.8249 - val_loss: 0.8949 - val_accuracy: 0.8191
Epoch 6/50
106/106 [==============================] - 0s 2ms/step - loss: 0.0201 - accuracy: 0.8214 - val_loss: 0.9173 - val_accuracy: 0.7899
Epoch 7/50
106/106 [==============================] - 0s 2ms/step - loss: 0.0117 - accuracy: 0.8409 - val_loss: 1.0631 - val_accuracy: 0.7686
Epoch 

### Utilizando XGBoosting for multilabel:

### Utilizando PyCaret:

In [67]:
import pycaret.classification
import pycaret.regression
import pycaret.clustering

> Adjustin Y_train for use with PyCaret:

In [84]:
df = X_train.copy()

In [81]:
# Definir uma função para mapear os valores das colunas para os rótulos desejados
def map_labels(row):
    if row['F_Objectivity'] == 0 and row['F_Subjectivity'] == 1:
        return 'Latent'
    elif row['F_Objectivity'] == 0 and row['F_Subjectivity'] == 0:
        return 'Contextual'
    elif row['F_Objectivity'] == 1 and row['F_Subjectivity'] == 0:
        return 'Manifest'
    elif row['F_Objectivity'] == 1 and row['F_Subjectivity'] == 1:
        return 'Perceptual'

In [86]:
# Aplicar a função de mapeamento para criar a nova coluna "Target"
df['target'] = Y_train.apply(map_labels, axis=1)

In [71]:
## Adding F_Subjectivity and F_Objectivity to df:
df['F_Subjectivity'] = Y_train['F_Subjectivity']
df['F_Objectivity'] = Y_train['F_Objectivity']

In [72]:
df['target'] = Y_train['F_Subjectivity']

In [87]:
# Exibir as primeiras linhas do DataFrame resultante para verificar
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,target
periodical,-1.627200,-0.782380,1.666100,0.64415,5.80250,-3.86060,2.114500,2.24640,0.11389,-0.57345,...,-1.15170,-1.55030,-2.03400,-2.33070,-0.35606,0.24120,-5.17900,-3.23970,-1.40730,Contextual
intellect,-0.154690,1.676800,-0.351630,-0.57609,2.08870,2.61040,0.664950,2.22990,-4.99240,-1.65200,...,-1.79910,-1.50010,-3.83950,1.14140,0.38565,1.78360,-0.12766,-5.47600,5.23490,Latent
culture,-0.025891,-2.009300,0.494080,-0.33639,5.29400,-0.38846,3.178800,2.94110,-2.58270,-1.90800,...,-0.75585,-0.47891,-2.78880,-0.65771,-4.51980,1.24270,-1.25050,-7.25270,5.32900,Latent
emancipation,-3.593900,-0.710900,2.035000,2.18820,2.32750,0.66056,2.739500,0.42427,1.17990,-0.65433,...,-0.22063,-1.37600,2.01930,1.47420,-0.93661,0.33936,-4.58320,-1.69060,-0.82634,Contextual
chasm,2.750600,2.573200,1.016900,-2.14450,-1.06080,1.80230,3.076300,3.05770,-1.46700,1.42200,...,0.71314,3.44560,-1.67040,2.73970,1.79250,-1.17130,0.82591,1.09540,-0.33145,Latent
legend,1.829300,-1.725200,5.139400,-2.16050,-1.08390,0.60729,4.525700,1.51890,5.95010,-1.85310,...,-0.35684,2.84170,2.21390,-0.22289,-2.05520,-0.70992,3.70210,-2.16260,-0.92589,Latent
decree,-3.328200,-2.839100,-1.438000,0.24277,2.37090,-2.00500,0.585080,0.32332,3.66520,-2.53490,...,5.59710,-1.19440,1.43720,1.87810,2.12330,2.14120,-0.40861,1.23430,1.27870,Contextual
linear,-3.708500,4.774500,-2.174300,2.81540,3.54900,-1.68670,2.050500,4.01830,-0.76764,1.34780,...,2.44370,4.66000,-2.26290,1.16150,-1.22950,-0.43797,-1.67510,-3.78960,-1.77310,Contextual
arse,2.075700,-0.026898,-0.060543,2.45900,-2.39870,1.23460,-3.563400,-0.69466,-0.87055,0.73186,...,0.49877,3.12810,-2.81070,-1.10680,2.30140,-0.60199,1.37360,0.26794,3.28290,Perceptual
jaguar,0.584360,2.370500,-1.796000,-2.67640,-0.31655,0.94561,-0.086297,0.01113,2.46700,1.80370,...,0.27040,3.28650,0.46756,1.35440,1.62060,-1.56070,-0.99583,-4.09790,-1.03070,Perceptual


In [74]:
## Descartando a última coluna:
df_last = df.iloc[:,:-1]
df_last.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,target,F_Subjectivity
periodical,-1.6272,-0.78238,1.6661,0.64415,5.8025,-3.8606,2.1145,2.2464,0.11389,-0.57345,5.7366,2.6897,-2.9695,2.0592,1.1753,2.1266,4.7035,2.7336,-3.5872,1.1187,-0.33484,-1.4071,-4.7561,2.2302,-2.059,-0.14411,-1.9042,-3.213,-0.34878,1.1597,-1.3406,0.47455,0.76463,0.62292,-2.3959,-0.086495,1.1585,-0.68091,1.1494,1.3205,1.8991,-0.035937,-0.36217,2.4583,1.1064,2.3078,-0.64062,0.28873,-1.6005,-2.3947,-2.2164,3.735,-0.78451,-0.50572,-2.4039,1.9877,-2.2064,2.1254,2.0112,-3.4274,0.38636,-0.99008,-2.2497,1.6211,3.0944,0.64471,1.4321,-0.62388,0.8677,1.1196,-1.9251,-2.9001,-2.8303,-2.8833,-0.33352,3.1787,-1.475,1.1811,-1.86,0.12378,-1.6496,-0.14419,-0.56384,1.1399,2.2299,3.6695,-1.0809,-2.7011,2.3617,-2.0884,-0.71105,1.6108,3.7237,-1.2607,2.374,-1.998,1.72,1.4512,-0.39968,4.1796,2.6991,2.5275,2.0868,1.2196,-1.0239,3.438,3.123,-3.8607,1.4855,-4.1848,3.7903,0.88335,-2.853,-0.98828,1.9463,1.1769,-1.8306,2.754,-1.7812,-1.4768,0.84724,-4.4108,1.7444,1.4066,1.8559,-4.2074,-0.42883,-2.2853,2.6108,-2.4452,-1.5658,0.6037,2.0717,2.7407,0.50984,0.82838,-1.9149,-1.2288,0.70129,-3.165,-0.40373,-1.5314,-0.56426,1.6354,3.8474,-0.94437,-0.99321,-0.052637,0.75979,3.275,2.3954,0.05155,0.77701,1.3314,-1.7912,0.54084,-0.39345,-0.94512,-1.1933,-1.5246,-1.9794,1.4443,1.1399,3.4458,-2.8027,-1.2135,-6.3629,1.6438,-0.24045,0.16754,1.4184,0.92152,2.7786,0.41264,2.3382,3.1647,1.2647,1.1072,-3.2381,-1.2591,-0.83185,-1.6059,0.57993,-2.118,-0.54878,0.76078,-0.074174,0.30904,-0.28318,1.7597,-0.83301,-2.5778,-3.7242,0.58209,3.8249,1.4794,-2.242,-2.0715,0.82211,3.4849,-1.6172,-4.0141,-0.16727,-2.0282,7.0865,0.84765,-3.4945,1.2765,2.2956,0.47523,2.3143,-0.93912,0.030626,-1.4647,-0.42383,1.7943,-0.78176,-3.8079,0.45426,0.58407,-0.15175,2.4248,-3.8759,-0.19335,-3.5155,-1.3907,-1.2319,0.48635,1.6602,0.84975,4.6175,-0.81367,0.47439,1.6774,1.7393,4.3165,0.14046,1.4182,-1.8713,-1.4948,-1.0273,-1.264,2.5117,2.1463,-1.052,2.5754,-2.8909,2.0706,-0.56698,2.6587,2.5999,-4.9548,-2.0778,-0.67968,-0.62967,-2.5925,-1.5708,-2.252,-0.10327,1.2572,-2.7904,2.8963,3.1598,2.2888,3.2515,-0.4181,2.1012,2.7386,-0.1432,-0.35501,-1.7387,1.5427,1.7338,-2.4797,1.0412,1.8452,-0.01234,-1.6399,-1.29,-0.055819,3.7824,0.87042,3.8564,0.43643,0.13927,-2.9037,0.46992,0.76273,0.52487,-2.5186,2.5555,-1.1517,-1.5503,-2.034,-2.3307,-0.35606,0.2412,-5.179,-3.2397,-1.4073,0,0


In [75]:
df_scnd_last = df.iloc[:, :-2].join(df.iloc[:, -1])
df_scnd_last.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,target,F_Objectivity
periodical,-1.6272,-0.78238,1.6661,0.64415,5.8025,-3.8606,2.1145,2.2464,0.11389,-0.57345,5.7366,2.6897,-2.9695,2.0592,1.1753,2.1266,4.7035,2.7336,-3.5872,1.1187,-0.33484,-1.4071,-4.7561,2.2302,-2.059,-0.14411,-1.9042,-3.213,-0.34878,1.1597,-1.3406,0.47455,0.76463,0.62292,-2.3959,-0.086495,1.1585,-0.68091,1.1494,1.3205,1.8991,-0.035937,-0.36217,2.4583,1.1064,2.3078,-0.64062,0.28873,-1.6005,-2.3947,-2.2164,3.735,-0.78451,-0.50572,-2.4039,1.9877,-2.2064,2.1254,2.0112,-3.4274,0.38636,-0.99008,-2.2497,1.6211,3.0944,0.64471,1.4321,-0.62388,0.8677,1.1196,-1.9251,-2.9001,-2.8303,-2.8833,-0.33352,3.1787,-1.475,1.1811,-1.86,0.12378,-1.6496,-0.14419,-0.56384,1.1399,2.2299,3.6695,-1.0809,-2.7011,2.3617,-2.0884,-0.71105,1.6108,3.7237,-1.2607,2.374,-1.998,1.72,1.4512,-0.39968,4.1796,2.6991,2.5275,2.0868,1.2196,-1.0239,3.438,3.123,-3.8607,1.4855,-4.1848,3.7903,0.88335,-2.853,-0.98828,1.9463,1.1769,-1.8306,2.754,-1.7812,-1.4768,0.84724,-4.4108,1.7444,1.4066,1.8559,-4.2074,-0.42883,-2.2853,2.6108,-2.4452,-1.5658,0.6037,2.0717,2.7407,0.50984,0.82838,-1.9149,-1.2288,0.70129,-3.165,-0.40373,-1.5314,-0.56426,1.6354,3.8474,-0.94437,-0.99321,-0.052637,0.75979,3.275,2.3954,0.05155,0.77701,1.3314,-1.7912,0.54084,-0.39345,-0.94512,-1.1933,-1.5246,-1.9794,1.4443,1.1399,3.4458,-2.8027,-1.2135,-6.3629,1.6438,-0.24045,0.16754,1.4184,0.92152,2.7786,0.41264,2.3382,3.1647,1.2647,1.1072,-3.2381,-1.2591,-0.83185,-1.6059,0.57993,-2.118,-0.54878,0.76078,-0.074174,0.30904,-0.28318,1.7597,-0.83301,-2.5778,-3.7242,0.58209,3.8249,1.4794,-2.242,-2.0715,0.82211,3.4849,-1.6172,-4.0141,-0.16727,-2.0282,7.0865,0.84765,-3.4945,1.2765,2.2956,0.47523,2.3143,-0.93912,0.030626,-1.4647,-0.42383,1.7943,-0.78176,-3.8079,0.45426,0.58407,-0.15175,2.4248,-3.8759,-0.19335,-3.5155,-1.3907,-1.2319,0.48635,1.6602,0.84975,4.6175,-0.81367,0.47439,1.6774,1.7393,4.3165,0.14046,1.4182,-1.8713,-1.4948,-1.0273,-1.264,2.5117,2.1463,-1.052,2.5754,-2.8909,2.0706,-0.56698,2.6587,2.5999,-4.9548,-2.0778,-0.67968,-0.62967,-2.5925,-1.5708,-2.252,-0.10327,1.2572,-2.7904,2.8963,3.1598,2.2888,3.2515,-0.4181,2.1012,2.7386,-0.1432,-0.35501,-1.7387,1.5427,1.7338,-2.4797,1.0412,1.8452,-0.01234,-1.6399,-1.29,-0.055819,3.7824,0.87042,3.8564,0.43643,0.13927,-2.9037,0.46992,0.76273,0.52487,-2.5186,2.5555,-1.1517,-1.5503,-2.034,-2.3307,-0.35606,0.2412,-5.179,-3.2397,-1.4073,0,0


### Pycaret Regression: 300 inputs and 1 numerical outputs

In [88]:
exp_regr = pycaret.regression.setup(df, target='target', session_id=123)

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Regression
3,Original data shape,"(3757, 301)"
4,Transformed data shape,"(3757, 301)"
5,Transformed train set shape,"(2629, 301)"
6,Transformed test set shape,"(1128, 301)"
7,Numeric features,300
8,Preprocess,True
9,Imputation type,simple


In [89]:
exp_regr.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:12:18
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


[]

In [100]:
exp_regr.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,02:34:55
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0330
en,Elastic Net,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0300
dummy,Dummy Regressor,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0300
llar,Lasso Least Angle Regression,0.1549,0.0350,0.1869,-0.0030,0.1311,0.5698,0.0280
br,Bayesian Ridge,0.1552,0.0351,0.1873,-0.0076,0.1314,0.5705,0.0320
omp,Orthogonal Matching Pursuit,0.1552,0.0352,0.1875,-0.0094,0.1315,0.5700,0.0290
lr,Linear Regression,0.1562,0.0357,0.1887,-0.0228,0.1324,0.5732,0.0310
lar,Least Angle Regression,0.1560,0.0357,0.1887,-0.0233,0.1324,0.5724,0.0290
ridge,Ridge Regression,0.1562,0.0357,0.1888,-0.0239,0.1325,0.5725,0.0300
ada,AdaBoost Regressor,0.1579,0.0360,0.1895,-0.0318,0.1339,0.6074,0.1460


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000, positive=False,
      precompute=False, random_state=123, selection='cyclic', tol=0.0001,
      warm_start=False)

### Pycaret Clustering

In [ ]:
exp_cluster = pycaret.clustering.setup()

### Pycaret Classification

In [90]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,target
periodical,-1.627200,-0.782380,1.666100,0.64415,5.802500,-3.86060,2.11450,2.246400,0.11389,-0.57345,...,-1.15170,-1.55030,-2.03400,-2.33070,-0.35606,0.241200,-5.179000,-3.23970,-1.40730,Contextual
intellect,-0.154690,1.676800,-0.351630,-0.57609,2.088700,2.61040,0.66495,2.229900,-4.99240,-1.65200,...,-1.79910,-1.50010,-3.83950,1.14140,0.38565,1.783600,-0.127660,-5.47600,5.23490,Latent
culture,-0.025891,-2.009300,0.494080,-0.33639,5.294000,-0.38846,3.17880,2.941100,-2.58270,-1.90800,...,-0.75585,-0.47891,-2.78880,-0.65771,-4.51980,1.242700,-1.250500,-7.25270,5.32900,Latent
emancipation,-3.593900,-0.710900,2.035000,2.18820,2.327500,0.66056,2.73950,0.424270,1.17990,-0.65433,...,-0.22063,-1.37600,2.01930,1.47420,-0.93661,0.339360,-4.583200,-1.69060,-0.82634,Contextual
chasm,2.750600,2.573200,1.016900,-2.14450,-1.060800,1.80230,3.07630,3.057700,-1.46700,1.42200,...,0.71314,3.44560,-1.67040,2.73970,1.79250,-1.171300,0.825910,1.09540,-0.33145,Latent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
prawn,-1.104300,-2.707300,-0.040561,0.26686,-0.001637,-2.23060,-0.22431,0.000931,-3.39360,0.79063,...,-2.56200,1.02140,-2.52040,-1.98950,1.48670,1.859700,-0.720330,-1.17220,4.63340,Manifest
tweezers,-1.051400,2.791100,1.661000,2.09520,-0.197260,-1.18820,-2.49820,0.762750,-3.10200,0.28440,...,-0.44270,1.51110,0.88290,-2.94660,0.53499,0.968740,-0.909550,-0.77088,-1.42830,Manifest
university,-1.409000,0.093909,0.172970,-0.25512,2.648800,2.68320,3.31220,3.572100,-0.04109,0.20994,...,-1.17040,-1.25060,-0.83331,0.25252,0.19663,1.934500,-0.546330,0.59882,0.67005,Perceptual
wasteful,-1.105000,-0.335100,-1.827500,-3.49660,1.209300,1.20280,0.13930,-1.561000,-0.51685,-0.91023,...,-0.92690,2.68170,-0.71914,-1.27960,0.42766,-1.082500,0.035378,-3.72800,1.17320,Latent


In [91]:
## Fazendo um experimento:
exp_class = pycaret.classification.setup(df, target='target', session_id=9088)

,Description,Value
0,Session id,9088
1,Target,target
2,Target type,Multiclass
3,Target mapping,"Contextual: 0, Latent: 1, Manifest: 2, Perceptual: 3"
4,Original data shape,"(3757, 301)"
5,Transformed data shape,"(3757, 301)"
6,Transformed train set shape,"(2629, 301)"
7,Transformed test set shape,"(1128, 301)"
8,Numeric features,300
9,Preprocess,True


In [92]:
exp_class.compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,03:13:12
Status,. . . . . . . . . . . . . . . . . .,Initializing CV
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6736,0.8904,0.6736,0.6632,0.6546,0.5408,0.5483,2.6710
lda,Linear Discriminant Analysis,0.6721,0.8801,0.6721,0.6697,0.6691,0.5483,0.5493,0.0640
ridge,Ridge Classifier,0.6691,0.0000,0.6691,0.6599,0.6558,0.5373,0.5421,0.0260
gbc,Gradient Boosting Classifier,0.6596,0.8855,0.6596,0.6462,0.6442,0.5234,0.5286,15.6720
lr,Logistic Regression,0.6489,0.8580,0.6489,0.6482,0.6466,0.5174,0.5185,0.3770
rf,Random Forest Classifier,0.6337,0.8569,0.6337,0.6165,0.5929,0.4764,0.4933,0.4090
knn,K Neighbors Classifier,0.6329,0.8297,0.6329,0.6184,0.6046,0.4800,0.4909,0.0300
svm,SVM - Linear Kernel,0.6314,0.0000,0.6314,0.6287,0.6243,0.4911,0.4941,0.0590
et,Extra Trees Classifier,0.6250,0.8640,0.6250,0.6174,0.5649,0.4577,0.4844,0.1080
qda,Quadratic Discriminant Analysis,0.6185,0.7886,0.6185,0.6541,0.5176,0.4410,0.4861,0.0410


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=9088, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

#### New Test

In [130]:
## Selecionar apenas os elementos do SCA que possuem fatores < 0.25 e > 0.75:
new_SCA_words = [word for word, subjectivity, objectivity in zip(df_factors['words'], df_factors['F_Subjectivity'], df_factors['F_Objectivity']) if (subjectivity > 0.75 or subjectivity < 0.25) and (objectivity > 0.75 or objectivity < 0.25)]


In [132]:
len(new_SCA_words)

851

In [133]:
new_SCA_embedding_rows =  nlp.vocab.vectors.find(keys=new_SCA_words)

In [134]:
len(new_SCA_embedding_rows)

851

> Separating the SCA-GlasgowNorms data into train and test:

In [ ]:
# Separar os dados em conjuntos de treino (70%) e teste (30%)
train_df, test_df = train_test_split(df_factors, test_size=0.2, random_state=42)

In [ ]:
# Função para criar os conjuntos de treino e resposta
def create_data(dataframe):
    X = {}
    Y = {}
    
    for index, row in dataframe.iterrows():
        word = row['words']
        f_objectivity = row['F_Objectivity']
        f_subjectivity = row['F_Subjectivity']

        if word in nlp.vocab:
            indice = nlp.vocab.strings[word]
            vetor_embedding = word_embedding[index]
            X[word] = vetor_embedding
            Y[word] = {'F_Objectivity': f_objectivity, 'F_Subjectivity': f_subjectivity}

    return pd.DataFrame.from_dict(X, orient='index'), pd.DataFrame.from_dict(Y, orient='index')

In [ ]:
# Creating train and test datasets:
X_train, Y_train = create_data(train_df)
X_test, Y_test = create_data(test_df)

# Exibir as dimensões dos conjuntos de treino e teste
print("Train data dimension:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

print("\nTest data dimension:")
print("X_test:", X_test.shape)
print("Y_test:", Y_test.shape)

Train data dimension:
X_train: (3757, 300)
Y_train: (3757, 2)

Test data dimension:
X_test: (923, 300)
Y_test: (923, 2)
